<a href="https://colab.research.google.com/github/casual-lab/MLC-Learn/blob/main/chapter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!python3 -m pip install mlc-ai-nightly -f https://mlc.ai/wheels

Looking in links: https://mlc.ai/wheels
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.6/185.6 MB 6.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import tvm
from tvm.ir.module import IRModule
from tvm.script import tir as T